In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

try:
    from pdtr import ParseDecisionTreeRules
except ModuleNotFoundError:
    import sys
    
    sys.path.append("../")
    from pdtr import ParseDecisionTreeRules
    
np.random.seed(1)

In [2]:
feature_map = {}
n_samples = 10000
ab = np.array(list('ABCDEFG'))

data = pd.DataFrame({
    'A': np.random.randint(10, size = n_samples),
    'B': ab[np.random.choice(7, n_samples)],
    'C': ab[np.random.choice(2, n_samples)],
    'D': np.random.random(size = n_samples),
    'target': np.random.randint(2, size = n_samples)
})

train, test = train_test_split(data, test_size=0.3, shuffle=data["target"])

In [3]:
pdtr_instance = ParseDecisionTreeRules(target="target", max_iter=8, output="model_report/决策树组合策略挖掘.xlsx")
pdtr_instance.fit(train, lift=0., max_depth=2, max_samples=1., verbose=False, max_features="auto")
all_rules = pdtr_instance.insert_all_rules(test=test)
pdtr_instance.save()

In [4]:
all_rules[0]

,组合策略,命中数,命中率,好样本数,好样本占比,坏样本数,坏样本占比,坏率,样本整体坏率,LIFT值
0,D > 0.999 & A <= 6.5,7,0.001000,7,0.001947,0,0.000000,0.000000,0.486286,0.000000
1,D <= 0.999 & D <= 0.999,6986,0.998000,3589,0.998053,3397,0.997944,0.486258,0.486286,0.999943
2,D <= 0.999 & D > 0.999,6,0.000857,0,0.000000,6,0.001763,1.000000,0.486286,2.056404
3,D > 0.999 & A > 6.5,1,0.000143,0,0.000000,1,0.000294,1.000000,0.486286,2.056404
4,C <= 0.486 & B <= 0.483,1552,0.221714,832,0.231368,720,0.211516,0.463918,0.486286,0.954002
5,C > 0.486 & B <= 0.499,3026,0.432286,1543,0.429088,1483,0.435664,0.490086,0.486286,1.007815
6,C <= 0.486 & B > 0.483,1967,0.281000,1002,0.278643,965,0.283490,0.490595,0.486286,1.008861
7,C > 0.486 & B > 0.499,455,0.065000,219,0.060901,236,0.069330,0.518681,0.486286,1.066618
8,A <= 5.5 & A > 3.5,1451,0.207286,769,0.213849,682,0.200353,0.470021,0.486286,0.966553
9,A <= 5.5 & A <= 3.5,2770,0.395714,1431,0.397942,1339,0.393361,0.483394,0.486286,0.994052


In [5]:
all_rules[1]

,组合策略,命中数,命中率,好样本数,好样本占比,坏样本数,坏样本占比,坏率,样本整体坏率,LIFT值
0,D > 0.999 & A <= 6.5,1,0.000333,0,0.0,1,0.000685,1.0,0.486667,2.054795
1,D <= 0.999 & D <= 0.999,2999,0.999667,1540,1.0,1459,0.999315,0.486495,0.486667,0.999648
2,D <= 0.999 & D > 0.999,0,0.0,0,0.0,0,0.0,0.0,0.486667,0.0
3,D > 0.999 & A > 6.5,0,0.0,0,0.0,0,0.0,0.0,0.486667,0.0
4,C <= 0.486 & B <= 0.483,666,0.222,328,0.212987,338,0.231507,0.507508,0.486667,1.042824
5,C > 0.486 & B <= 0.499,1282,0.427333,667,0.433117,615,0.421233,0.479719,0.486667,0.985724
6,C <= 0.486 & B > 0.483,847,0.282333,441,0.286364,406,0.278082,0.479339,0.486667,0.984943
7,C > 0.486 & B > 0.499,205,0.068333,104,0.067532,101,0.069178,0.492683,0.486667,1.012362
8,A <= 5.5 & A > 3.5,604,0.201333,319,0.207143,285,0.195205,0.471854,0.486667,0.969564
9,A <= 5.5 & A <= 3.5,1162,0.387333,591,0.383766,571,0.391096,0.491394,0.486667,1.009714
